In [1]:
import pandas as pd

In [2]:
ls

alannn.ipynb                     nets/
annotations/                     out.json
annotations_s/                   prepare-data-for-yolo-Copy1.ipynb
annotations_s.zip                prepare-data-for-yolo-Copy2.ipynb
classic.ipynb                    prepare-data-for-yolo.ipynb
confusion_matrix_normalized.png  __pycache__/
data.csv                         qa_test.ipynb
dert_kaggle.ipynb                req.txt
detect/                          runs/
detect2/                         static_quantized.onnx
detect3/                         subsamples.ipynb
detect4/                         testc.ipynb
detr.ipynb                       test.json
detr.py                          test.png
donuts-found.jpg                 test_qa.ipynb
dynamic_quantized.onnx           test.yaml
eda.ipynb                        tranformers.py
evaluate.py                      utils/
hw_02_pruning_solution.ipynb     weights/
img/                             yolo.ipynb
img.zip                          yolo-second-epoch.i

In [3]:
df = pd.read_csv('data.csv')
df

,img_id,cam_id,seq_id,frame_id,bbox_id,x_min,x_max,y_min,y_max,label_l1,label_l2,fao_mfa,train,val,test
0,da7c5ef0-db28-11ea-b693-9b3523cc10a5,60,13209,33,1,626,754,-1,88,Human,HUMAN,81,True,False,False
1,da7c5ef0-db28-11ea-b693-9b3523cc10a5,60,13209,33,3,544,710,15,203,Human,HUMAN,81,True,False,False
2,da7c5ef0-db28-11ea-b693-9b3523cc10a5,60,13209,33,4,629,735,210,562,Human,HUMAN,81,True,False,False
3,da7c5ef0-db28-11ea-b693-9b3523cc10a5,60,13209,33,5,615,883,550,907,Human,HUMAN,81,True,False,False
4,da7c5ef0-db28-11ea-b693-9b3523cc10a5,60,13209,33,6,885,962,826,875,Unknown,OTH,81,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549204,fcdecc18-5952-11ec-a87d-47788f674eed,93,15,28,126812,227,510,702,992,Yellowfin tuna,YFT,81,False,False,True
549205,fcdeef2c-5952-11ec-a8ae-cf526681fdda,93,15,77,128339,223,518,700,996,Yellowfin tuna,YFT,81,False,False,True
549206,fcdeef2c-5952-11ec-a8ae-cf526681fdda,93,15,77,128338,875,1127,668,1030,Yellowfin tuna,YFT,81,False,False,True
549207,fcdeef2c-5952-11ec-a8ae-cf526681fdda,93,15,77,128337,1069,1465,487,674,Yellowfin tuna,YFT,81,False,False,True


In [4]:
test = df[df['test'] == True]
val = df[df['val'] == True]


In [5]:
test_img = list(set(test['img_id']))[:1000]
val_img = list(set(val['img_id']))[:1000]


In [6]:
val = val[val['img_id'].isin(val_img)]
test = test[test['img_id'].isin(test_img)]

In [7]:
ls annotations/images

test/  train/  val/


In [23]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
from tqdm import tqdm
image_path = 'img/'

In [10]:
t = '''
  0: Long snouted lancetfish
  1: Roudie scolar
  2: Marlin
  3: Swordfish
  4: Great barracuda
  5: Thresher shark
  6: Lancetfish
  7: Pomfret
  8: Tuna
  9: Opah
  10: Pelagic stingray
  11: Mahi mahi
  12: Striped marlin
  13: Wahoo
  14: No fish
  15: Human
  16: Skipjack tuna
  17: Yellowfin tuna
  18: Unknown
  19: Blue marlin
  20: Oilfish
  21: Bigeye tuna
  22: Snake mackerel
  23: Indo Pacific sailfish
  24: Water
  25: Sickle pomfret
  26: Albacore
  27: Brama
  28: Black marlin
  29: Shortbill spearfish
  30: Mola mola
  31: Rainbow runner
  32: Escolar
  33: Shark
'''

In [25]:
map_ids = {}
for i in t.split('\n')[1:-1]:
    id_int , s = i.split(':')
    map_ids[int(id_int)]=s[1:]
map_ids = {v:k for k,v in map_ids.items()}

In [26]:
# import json 
# with open('map_ids.json','r') as f:
#     map_ids =json.load(f)

In [27]:
from  shutil import copy

In [28]:
os.makedirs('annotations_s', exist_ok=True)
os.makedirs('annotations_s/images', exist_ok=True)
os.makedirs('annotations_s/images/train', exist_ok=True)
os.makedirs('annotations_s/images/val', exist_ok=True)
os.makedirs('annotations_s/images/test', exist_ok=True)
os.makedirs('annotations_s/labels', exist_ok=True)
os.makedirs('annotations_s/labels/train', exist_ok=True)
os.makedirs('annotations_s/labels/val', exist_ok=True)
os.makedirs('annotations_s/labels/test', exist_ok=True)

for idx, row in tqdm(test.iterrows(),total=len(test)):
    img_path = f"{image_path}/{row['img_id']}.jpg"
    
  
    img = cv2.imread(img_path)
    h, w, _ = img.shape  
    x_center = (row['x_min'] + row['x_max']) / 2 / w
    y_center = (row['y_min'] + row['y_max']) / 2 / h
    bbox_width = (row['x_max'] - row['x_min']) / w
    bbox_height = (row['y_max'] - row['y_min']) / h

    class_id = map_ids[row['label_l1']]
    
    txt_file_path = os.path.join('annotations_s/labels/train', os.path.splitext(os.path.basename(img_path))[0] + '.txt')
    img_file_path =  os.path.join('annotations_s/images/train', os.path.splitext(os.path.basename(img_path))[0] + '.jpg')
    with open(txt_file_path, 'a') as f:
        f.write(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}\n")
    copy(img_path,img_file_path)



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3785/3785 [00:13<00:00, 281.58it/s]


In [29]:
for idx, row in tqdm(val.iterrows(),total=len(val)):
    img_path = f"{image_path}/{row['img_id']}.jpg"
    
    img = cv2.imread(img_path)
    h, w, _ = img.shape  

    x_center = (row['x_min'] + row['x_max']) / 2 / w
    y_center = (row['y_min'] + row['y_max']) / 2 / h
    bbox_width = (row['x_max'] - row['x_min']) / w
    bbox_height = (row['y_max'] - row['y_min']) / h

    class_id = map_ids[row['label_l1']]
    
    txt_file_path = os.path.join('annotations_s/labels/val', os.path.splitext(os.path.basename(img_path))[0] + '.txt')
    img_file_path =  os.path.join('annotations_s/images/val', os.path.splitext(os.path.basename(img_path))[0] + '.jpg')
    with open(txt_file_path, 'a') as f:
        f.write(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}\n")
    copy(img_path,img_file_path)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3699/3699 [00:14<00:00, 257.36it/s]
